<a id='toc'></a>
# Table of Contents:
1. [Make Graph](#makegraph)
2. [Read in Yearly Prediction and Scale Back to Original Interval](#readscale)
3. [Exploratory Data Analysis](#eda)
4. [1D CNN](#1dcnn)         <br>

# Leak Detection

> Garðar Örn Garðarsson <br>
Integrated Machine Learning Systems 20-21 <br>
University College London

<a id='makegraph'></a>
*Back to [Table of Contents](#toc)*

## 1. Make Graph

Convert the `EPANET` model to a `networkx` graph

In [1]:
import os
import yaml
import time
import torch
import epynet
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from utils.epanet_loader import get_nx_graph
from utils.epanet_simulator import epanetSimulator
from utils.data_loader import battledimLoader, dataCleaner, dataGenerator, embedSignalOnGraph, rescaleSignal
from modules.torch_gnn import ChebNet
from utils.visualisation import visualise

# Runtime configuration
path_to_wdn     = './data/L-TOWN.inp'
path_to_data    = './data/l-town-data/'
weight_mode     = 'pipe_length'
self_loops      = True
scaling         = 'minmax'
figsize         = (50,16)
print_out_rate  = 1               
model_name      = 'l-town-chebnet-' + weight_mode +'-' + scaling + '{}'.format('-self_loop' if self_loops else '')
last_model_path = './studies/models/' + model_name + '-1.pt'
last_log_path   = './studies/logs/'   + model_name + '-1.csv' 

# Import the .inp file using the EPYNET library
wdn = epynet.Network(path_to_wdn)

# Solve hydraulic model for a single timestep
wdn.solve()

# Convert the file using a custom function, based on:
# https://github.com/BME-SmartLab/GraphConvWat 
G , pos , head = get_nx_graph(wdn, weight_mode=weight_mode, get_head=True)

<a id='readscale'></a>
*Back to [Table of Contents](#toc)*

## 2. Read in Yearly Prediction and Scale Back to Original Interval

In [2]:
def read_prediction(filename='predictions.csv', scale=1, bias=0, start_date='2018-01-01 00:00:00'):
    df = pd.read_csv(filename, index_col='Unnamed: 0')
    df.columns = ['n{}'.format(int(node)+1) for node in df.columns]
    df = df*scale+bias
    df.index = pd.date_range(start=start_date,
                             periods=len(df),
                             freq = '5min')
    return df

In [92]:
n_timesteps = 3                                              # Timesteps, t-1, t-2...t-n used to predict pressure at t
sample_rate = 5                                              # Minutes sampling rate of data
offset      = pd.DateOffset(minutes=sample_rate*n_timesteps) # We require n_timesteps of data our first prediction

Load predictions

In [93]:
p18 = read_prediction(filename='2019_predictions.csv',
                      start_date=pd.Timestamp('2019-01-01 00:00:00')+offset)

Load reconstructions

In [94]:
r18 = read_prediction(filename='2019_reconstructions.csv',
                      start_date='2019-01-01 00:00:00')

Load leakage dataset

In [6]:
l18 = pd.read_csv('data/l-town-data/2018_Leakages.csv',decimal=',',sep=';',index_col='Timestamp')
l18.index = r18.index # Fix the index column timestamp format

<a id='eda'></a>
*Back to [Table of Contents](#toc)*

## 3. Exploratory Data Analysis

### 3.1 Data Wrangling

Lets create a dictionary of the format:

`{ 'pipe_name' : [ connected_node_1 , connected_node_2 ] }`

For all the pipes in the network

In [7]:
neighbours_by_pipe = {}

for node in G:
    for neighbour, connecting_edge in G[node].items():
        if connecting_edge['name'] == 'SELF':
            continue
        else:
            neighbours_by_pipe[connecting_edge['name']] = [node, neighbour]
            

Let's also create the inverse, when we want to look up pipes by their connecting nodes

In [8]:
pipe_by_neighbours = { str(neighbour_list) : pipe for pipe , neighbour_list in neighbours_by_pipe.items()}

We'll also create a function to perform the lookup, as the order in which the nodes appear in the key matter and we don't bother with raising endless errors when looking up nodes we know to be connected, just cause we input them wrong

In [9]:
def pipeByneighbourLookup(node1, node2, pipe_by_neighbours):
    try:
        return pipe_by_neighbours[str([node1,node2])]    # If we don't find the first combination
    except:
        try:                                            # We try the next
            return pipe_by_neighbours[str([node2,node1])]
        except:                                         # And if we still don't find it
            return None                                 # We return nothing

Try it out:

In [10]:
pipeByneighbourLookup(1,347,pipe_by_neighbours)

'p253'

In [11]:
pipeByneighbourLookup(347,1,pipe_by_neighbours)

'p253'

In [12]:
pipeByneighbourLookup(1,2,pipe_by_neighbours)

These might really come in handy when it comes to looking up pipe by node or for converting pipe leakage dataframes to leaky nodes!

### 3.2 Calculating Per-Node Error

We may calculate the node-wise validation error, $\epsilon_{n_{i}}$, by subtracting the predicted values with the reconstructed ones.

In [95]:
error_by_node = (p18-r18).copy()

### 3.3 Calculating Per-Pipe Error

We may calculate the pipe-wise validation errors, $\epsilon_{p_i}$, as the difference of the validation error of the two nodes connecting the pipe

In [96]:
error_by_pipe = {}

for key,value in neighbours_by_pipe.items():
    node_1 = 'n' + str(value[0])
    node_2 = 'n' + str(value[1])
    error_by_pipe[key] = ( error_by_node[node_1] - error_by_node[node_2] )
    
error_by_pipe = pd.DataFrame(error_by_pipe)

### 3.4 Leakage Labelset

Make a complete leakage labelset for each pipe

In [83]:
leaks_by_pipe = pd.DataFrame([], index=error_by_pipe.index, columns=error_by_pipe.columns)

for leaky_pipe in l18:
    leaks_by_pipe[leaky_pipe] = l18[leaky_pipe]
    
leaks_by_pipe = leaks_by_pipe.fillna(0.0)

Keep a dictionary of the timestamps of leakages

In [17]:
leak_timestamps = {}

for leak in l18:
    leak_timestamps[leak] = l18.index[l18[leak]>0]

In [18]:
leak_timestamps.keys()

dict_keys(['p31', 'p158', 'p183', 'p232', 'p257', 'p369', 'p427', 'p461', 'p538', 'p628', 'p654', 'p673', 'p810', 'p866'])

In [19]:
leak_timestamps['p31'][:3]

DatetimeIndex(['2018-06-29 01:35:00', '2018-06-29 01:40:00',
               '2018-06-29 01:45:00'],
              dtype='datetime64[ns]', freq='5T')

In [20]:
neighbours_by_pipe['p31']

[42, 40]

Alternatively, make a node-wise leak labelset!

In [21]:
leaks_by_node = pd.DataFrame(data    = np.zeros(error_by_node.shape), 
                             index   = error_by_node.index, 
                             columns = np.arange(1,783))

In [22]:
for pipe,neighbours in neighbours_by_pipe.items():
    for neighbour in neighbours:
        leaks_by_node[neighbour] += leaks_by_pipe[pipe]

In [23]:
leaks_by_node[40][leak_timestamps[pipeByneighbourLookup(40,42,pipe_by_neighbours)]]

2018-06-29 01:35:00     0.01
2018-06-29 01:40:00     0.01
2018-06-29 01:45:00     0.01
2018-06-29 01:50:00     0.01
2018-06-29 01:55:00     0.01
                       ...  
2018-08-12 17:10:00    16.01
2018-08-12 17:15:00    16.01
2018-08-12 17:20:00    16.00
2018-08-12 17:25:00    15.99
2018-08-12 17:30:00    16.00
Freq: 5T, Name: 40, Length: 12864, dtype: float64

In [24]:
leaks_by_node

,1,2,3,4,5,6,7,8,9,10,...,773,774,775,776,777,778,779,780,781,782
2018-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 00:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 00:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 00:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 23:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-31 23:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-31 23:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-31 23:50:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 3.5 Dataset Pre-Processing

Process the dataset for timeseries classification. <br>
I want to devise a set where I have a configurable window of observations, `[1, 5, 10, ... 200]`.<br>
For that window of observations, I sum the per pipe events such that the according label will read `[0, 0, 1, 0, ...]`, where the set bit indicates the leaky pipes for that interval. <br>

First clean out the `NaNs` from the feature and labelset caused by the window size of the predictive model

In [97]:
error_by_pipe = error_by_pipe.dropna()

In [26]:
leaks_by_pipe = leaks_by_pipe[leaks_by_pipe.index.isin(error_by_pipe.index)]

Similarly for the pipe-wise classification, do it for the node-wise classification

In [27]:
error_by_node = error_by_node.dropna()

In [28]:
leaks_by_node = leaks_by_node[leaks_by_node.index.isin(error_by_node.index)]

Split up the feature and labelset for classification

In [98]:
error_by_pipe.to_csv('error_by_pipe_2019.csv')

In [71]:
leaks_by_pipe[leaks_by_pipe<4.5]=0
leaks_by_pipe[leaks_by_pipe>0  ]=1

In [77]:
leaks_by_pipe = leaks_by_pipe.astype('int')

In [91]:
leaks_by_pipe.to_csv('leaks_by_pipe_r.csv')

In [28]:
def classificationTaskSplitter(x, y, using_window=True, window_size=10, data_slice=None):
    
    if not data_slice:
        
        data_slice = len(x)
    
    features = []
    labels   = []
    
    if using_window:
        
        window_start = 0
        window_end   = window_start+window_size

        for i in range(len(x[window_start:window_start+data_slice-window_size])):
            features.append( x.iloc[window_start:window_end].to_numpy() )
            labels.append(  (y.iloc[window_start:window_end].sum().to_numpy() > 0).astype(int) )
            window_start += 1
            window_end   += 1
    
    else:
        
        for i in range(len(x[:data_slice-window_size])):
            features.append( x.iloc[i].to_numpy() )
            labels.append(  (y.iloc[i].to_numpy() > 0).astype(int) )
    
    return np.array(features), np.array(labels)

In [29]:
x, y = classificationTaskSplitter(x            = error_by_pipe, 
                                  y            = leaks_by_pipe, 
                                  using_window = True,
                                  window_size  = 24, 
                                  data_slice   = None)

In [30]:
print("x: {} \ny: {}".format(x.shape, y.shape))

x: (105093, 24, 905) 
y: (105093, 905)


In [31]:
task   = 'pipe'
window = 24
name   = task + '_window_' + str(window)

In [32]:
os.chdir('/Volumes/GoogleDrive/Drifið mitt/00_leak_detection')

In [33]:
np.save(name+'_x',x)
np.save(name+'_y',y)
#np.save(name+'idx',idx)

Create a random sampler

If we sample 10.000 pressure scenes (and their corresponding leak labels) from the 2018 data, that would amount to:

<a id='1dcnn'></a>
*Back to [Table of Contents](#toc)*

## 4. 1D-CNN